In [1]:
import msgspec
import cloudpickle as pickle
import typing

In [2]:
def create_struct(function: typing.Callable) -> msgspec.Struct:

    # Get standard type hints
    msg_spec_hints = typing.get_type_hints(function)

    assert "return" in msg_spec_hints and len(msg_spec_hints) > 1, \
        "Typehint must be specified for all arguments and return value."

    msg_spec_hints = list(msg_spec_hints.items())

    # Handle `return` type hint
    msg_spec_hints[-1] = tuple([msg_spec_hints[-1][0], typing.Optional[msg_spec_hints[-1][1]], None])

    # Create struct
    Function = msgspec.defstruct(
        "Function",
        msg_spec_hints,
    )

    return Function

def enc_hook(obj: typing.Any) -> bytes:
    return pickle.dumps(obj)

def dec_hook(type: typing.Any, obj: bytes) -> typing.Any:
    return pickle.loads(obj)

In [3]:
class TestClass:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

def test_function(a: int, b: str, c: typing.List[int]) -> TestClass:
    return TestClass(a, b, c)

MyStruct = create_struct(test_function)

encoder = msgspec.json.Encoder(enc_hook=enc_hook)
decoder = msgspec.json.Decoder(type=MyStruct, dec_hook=dec_hook)

my_struct = MyStruct(1, "2", [3, 4])
print(my_struct)

Function(a=1, b='2', c=[3, 4], return=None)


In [4]:
# encoder = msgspec.json.encode(my_struct)
encoded = encoder.encode(my_struct)
print(type(encoded))
print(encoded)

<class 'bytes'>
b'{"a":1,"b":"2","c":[3,4],"return":null}'


In [12]:
enc_array = encoded
print(type(enc_array))
print(enc_array[:16])

<class 'bytes'>
b'{"a":1,"b":"2","'


In [6]:
decoded = decoder.decode(encoded)
print(decoded)

Function(a=1, b='2', c=[3, 4], return=None)


In [7]:
# %time p_encoded = pickle.dumps(my_struct)
# %time p_decoded = pickle.loads(p_encoded)